In [1]:
# Import the necessary packages
import matplotlib.pyplot as plt
import numpy as np
import tidy3d as td
import tidy3d.web as web
import scienceplots

This is reproduction of the results in 1(b) using Diffraction Monitor. Parameters from [the paper](https://duke.box.com/s/xvysh66ew21g9o9e5pjcd11o11bzo73t):

$$P= 190nm$$
$$h= 230nm$$
$$D= [60, 135]nm$$
$$\lambda = 532nm$$

In [43]:
# 0 Define a FreqRange object with desired wavelengths
lda0 = 0.532
freq0 = td.C_0 / lda0
print("%E" % freq0)

5.635197E+14


In [44]:
# 1 Computational Domain Size
h = 0.230  # Height of cylinder
spc = 8
Lz = spc + h + h + spc

Px = Py = P = 0.190 # periodicity
sim_size = [Px, Py, Lz]

In [45]:
# 2 Grid Resolution
dl = P / 32
horizontal_grid = td.UniformGrid(dl=dl)
vertical_grid = td.AutoGrid(min_steps_per_wvl=32)
grid_spec=td.GridSpec(
    grid_x=horizontal_grid,
    grid_y=horizontal_grid,
    grid_z=vertical_grid,
)

In [46]:
sap = td.material_library['Sapphire']['Horiba']
si = td.material_library['cSi']['Green2008']

h = 0.230

substrate = td.Structure(
    geometry=td.Box(
        center=(0,0,-Lz/2),
        size=(td.inf,td.inf,2 * (spc+h))
    ),
    medium=sap,
    name='substrate'
)

In [47]:
source = td.PlaneWave(
    source_time=td.GaussianPulse(freq0=freq0, fwidth=20e12),
    size=(td.inf, td.inf,0),
    center=(0, 0, Lz/2 - spc + 0.5 * lda0),
    direction="-",
    pol_angle=0
)

In [48]:
monitor = td.DiffractionMonitor(
    center=(0, 0, -Lz/2 + spc - 0.5 * lda0),
    size=(td.inf,td.inf,0),
    freqs=[freq0],
    name='diffraction_monitor',
    normal_dir='-', # away from structure
)

In [49]:
bc = td.BoundarySpec(
    x=td.Boundary.periodic(), 
    y=td.Boundary.periodic(), 
    z=td.Boundary.pml()
)

In [67]:
D = np.arange(0.060, 0.140, 0.010)
print(D)
sims = {}
run_time = 50

def simulate(D):
    for i, diameter in enumerate(D):
        cylinder = td.Structure(
            geometry=td.Cylinder(
                center=[0, 0, h / 2], 
                radius=diameter/2, 
                length=h, 
            axis=2), 
            medium=si
        )
        
        sim = td.Simulation(
            size = sim_size,
            grid_spec = grid_spec,
            structures = [cylinder, substrate],
            sources=[source],
            monitors=[monitor],
            run_time= run_time / freq0,
            boundary_spec=bc
        )

        sims[f"d{i}"] = sim

[0.06 0.07 0.08 0.09 0.1  0.11 0.12 0.13 0.14]


In [68]:
simulate(D)
print(sims.keys())

dict_keys(['d0', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8'])
